<a href="https://colab.research.google.com/github/Idiallo/Projets-Computer-Vision-UVB/blob/main/analysesementiquechatsuspect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install emot


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 KB 4.3 MB/s eta 0:00:00


In [15]:
import numpy as np
import pandas as pd
import os

In [16]:
for dirname, _, filenames in os.walk('/content/sample_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/tweets_suspect.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_train_small.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_train.csv


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
from numpy.random import RandomState


df = pd.read_csv('../content/sample_data/tweets_suspect.csv')
rng = RandomState()

df['nonsuspect'] = df['label'].where(df['label'] == 0)
df['suspect'] = df['label'].where(df['label'] == 1)
df.to_csv("tweets_suspect.csv", index=False)

Je vais ajouter deux collones nonsuspect et suspect en vue spliter la collone label

In [19]:
cols_target = ['message','nonsuspect','suspect']

In [20]:
df.head()

,message,label,nonsuspect,suspect
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1,NaN,1.0
1,is upset that he can't update his Facebook by ...,1,NaN,1.0
2,@Kenichan I dived many times for the ball. Man...,1,NaN,1.0
3,my whole body feels itchy and like its on fire,0,0.0,NaN
4,"@nationwideclass no, it's not behaving at all....",1,NaN,1.0


ici je divise mon jeu de donnée en deux collones , a savoir suspect et non suspect.

In [21]:
df.isnull().sum()

message           0
label             0
nonsuspect    53855
suspect        6145
dtype: int64

In [22]:
df.describe()

,label,nonsuspect,suspect
count,60000.000000,6145.0,53855.0
mean,0.897583,0.0,1.0
std,0.303198,0.0,0.0
min,0.000000,0.0,1.0
25%,1.000000,0.0,1.0
50%,1.000000,0.0,1.0
75%,1.000000,0.0,1.0
max,1.000000,0.0,1.0


Je n'ai plus besoin de la colonne label 

In [23]:
df.drop(['label'],axis=1,inplace=True)

In [24]:
df.head()

,message,nonsuspect,suspect
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",NaN,1.0
1,is upset that he can't update his Facebook by ...,NaN,1.0
2,@Kenichan I dived many times for the ball. Man...,NaN,1.0
3,my whole body feels itchy and like its on fire,0.0,NaN
4,"@nationwideclass no, it's not behaving at all....",NaN,1.0


In [26]:
df = df.replace('',np.nan)
df.head()

,message,nonsuspect,suspect
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",NaN,1.0
1,is upset that he can't update his Facebook by ...,NaN,1.0
2,@Kenichan I dived many times for the ball. Man...,NaN,1.0
3,my whole body feels itchy and like its on fire,0.0,NaN
4,"@nationwideclass no, it's not behaving at all....",NaN,1.0


Je divise mon jeux de donnée en train et test

In [27]:
train = df.sample(frac=0.7, random_state=rng)
test = df.loc[~df.index.isin(train.index)]

In [28]:
train.shape

(42000, 3)

Je vais assainir le contenue des messages

In [29]:
import re
train["message"]=train["message"].apply(lambda x: x.lower())
train["message"]=train["message"].apply(lambda x: re.sub('\w*\d\w*','', x))
test.shape


(18000, 3)

In [22]:
import string
train["message"]=train["message"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [30]:
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    return text
train["message"] = train["message"].apply(clean_text)
train["message"]

42438    @bradhanks thanks for the #followfriday  how's...
8644           my computer is broken, technology hates me 
56975     queer prom tonite   soooooo much fun. i loved it
16780                               dammit....giants lose 
34769                                     happy birthday! 
                               ...                        
4186     @bradiewebbstack aww wish u talked to mee...mi...
30013    @ at least i won't be the only one feeling los...
45319                                       hershey park! 
34225         its , and im still not sleepy... feeel like 
23516    @andyclemmensen u had curry earlier too, has t...
Name: message, Length: 42000, dtype: object

In [31]:
train

,message,nonsuspect,suspect
42438,@bradhanks thanks for the #followfriday how's...,NaN,1.0
8644,"my computer is broken, technology hates me",0.0,NaN
56975,queer prom tonite soooooo much fun. i loved it,NaN,1.0
16780,dammit....giants lose,NaN,1.0
34769,happy birthday!,NaN,1.0
...,...,...,...
4186,@bradiewebbstack aww wish u talked to mee...mi...,NaN,1.0
30013,@ at least i won't be the only one feeling los...,NaN,1.0
45319,hershey park!,NaN,1.0
34225,"its , and im still not sleepy... feeel like",NaN,1.0


In [32]:
train.to_csv('train.csv',index=False)

Début de l'apprentissage

In [34]:
X = train.message
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect
type(X)



pandas.core.series.Series

In [35]:
test_df = test.message
X_train = vect.fit_transform(X.astype('U'))
X_test = vect.transform(test_df.astype('U'))


In [36]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=24.0)
submission_binary1 = pd.read_csv('./tweets_suspect.csv')
submission_binary1.shape

(60000, 4)

In [ ]:
from sklearn.metrics import classification_report
for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_train, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_train)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print(classification_report(y,y_pred_X))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(X_test)[:,1]
    print(test_y_prob.shape)
    submission_binary1[label] = test_y_prob

... Processing message
